In [1]:
import os
import pandas as pd
import gensim
import gensim.corpora as corpora
from sklearn.model_selection import train_test_split
from parameters import trainable_data_folder, num_topics

In [2]:
print("\nStarting to process files...")
i = 1
for f in os.listdir(trainable_data_folder):
    print(f" i = {i}")
    i+=1
    if f.startswith("3.") and f.endswith(".csv"): # Ensure it's a CSV from step 3
        print(f"\nProcessing file: {f}")
        try:

            parts = f.replace(".csv", "").split("_")
            if "assignees" not in parts:
                 print(f"  Skipping {f}: Filename does not match expected format for 'assignees'.")
                 continue

            assignees_index = parts.index("assignees")
            project_name = "_".join(parts[1:assignees_index-1]) # Handles project names with underscores
            num_assignees_str = parts[assignees_index-1]
            
            if not project_name or not num_assignees_str.isdigit():
                print(f"  Skipping {f}: Could not parse project_name or num_assignees from filename.")
                continue
            num_assignees = int(num_assignees_str)
            print(f"  Project: {project_name}, Num Assignees Config: {num_assignees}")
        except (IndexError, ValueError) as e:
            print(f"  Skipping {f}: Error parsing filename ({e}). Expected format like '3_PROJECT_N_assignees(_processed).csv'")
            continue
        # --- Step 3: Create Output Directory ---
        output_dir_name = f"4_{project_name}_{num_assignees}_assignees_features"
        full_output_path = os.path.join(trainable_data_folder, output_dir_name)
        os.makedirs(full_output_path, exist_ok=True)
        print(f"  Output directory: {full_output_path}")
        
        # --- Step 4: Load Data and Split ---
        input_file_path = os.path.join(trainable_data_folder, f)
        try:
            df = pd.read_csv(input_file_path, sep='\t', encoding='utf-8')
        except FileNotFoundError:
            print(f"  Error: File {input_file_path} not found. Skipping.")
            continue
        except Exception as e:
            print(f"  Error reading {input_file_path}: {e}. Skipping.")
            continue

        if 'assignee_id' not in df.columns:
            print(f"  Error: 'assignee_id' column not found in {f}. Skipping.")
            continue
        
        # Ensure 'title' and 'description' columns exist for LDA
        if 'processed_title' not in df.columns or 'processed_description' not in df.columns:
            print(f"  Error: 'title' or 'description' column not found in {f}. These are needed for LDA. Skipping.")
            continue

        y = df[["assignee_id"]]
        X = df.drop('assignee_id', axis=1) # Features

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, shuffle=True, random_state=42 # Added random_state
        )
        print(f"  Data loaded. Training set size: {len(X_train)}, Test set size: {len(X_test)}")

        # --- Step 5: Prepare Text Corpus for LDA (from Training Data) ---
        print("  Preparing LDA corpus from training data...")
        # Ensure columns are string type and handle potential NaN values before concatenation
        X_train_titles = X_train['processed_title'].fillna('').astype(str)
        X_train_descriptions = X_train['processed_description'].fillna('').astype(str)
        
        corpus_texts_train = (X_train_titles + ' ' + X_train_descriptions).to_list()
        corpus_tokens_train = [text.split() for text in corpus_texts_train]

        dictionary = corpora.Dictionary(corpus_tokens_train)
        # Filter out tokens that appear < 30 times, and tokens that appear in more than 20% of the documents
        dictionary.filter_extremes(no_below=30, no_above=0.2) 
        
        corpus_train_bow = [dictionary.doc2bow(text) for text in corpus_tokens_train]
        print(f"  Dictionary size after filtering: {len(dictionary)}")
        if not corpus_train_bow:
            print("  Warning: LDA training corpus is empty after filtering. Skipping LDA for this file.")
            continue

        # --- Step 6: Train LDA Model and Extract Topic Features ---
        for topics_k in num_topics:
            print(f"    Training LDA model with {topics_k} topics...")
            try:
                lda_model = gensim.models.ldamodel.LdaModel(
                    corpus=corpus_train_bow,
                    id2word=dictionary,
                    num_topics=topics_k,
                    random_state=100,
                    update_every=1,
                    alpha='auto',
                    per_word_topics=True
                )
            except Exception as e:
                print(f"    Error training LDA model with {topics_k} topics: {e}. Skipping this topic number.")
                continue

            print(f"    Extracting features for {topics_k} topics...")
            for dataset_name, current_x_data_original in [("X_train", X_train), ("X_test", X_test)]:
                # Work on a copy to avoid modifying X_train/X_test in place if issues occur
                current_x_data = current_x_data_original.copy()

                topic_per_issue_list = []
                topterms_per_issue_list = []
                distribution_per_issue_list = []

                # Ensure columns are string type and handle potential NaN
                current_x_titles = current_x_data['processed_title'].fillna('').astype(str)
                current_x_descriptions = current_x_data['processed_description'].fillna('').astype(str)

                for index in range(len(current_x_data)):
                    # Use .iloc for row-wise access based on integer position if needed, or combine series directly
                    issue_text_combined = current_x_titles.iloc[index] + ' ' + current_x_descriptions.iloc[index]
                    issue_tokens = issue_text_combined.split()
                    issue_bow = dictionary.doc2bow(issue_tokens)
                    
                    # Get raw topics, ensuring a score for all topics even if 0
                    raw_doc_topics = lda_model.get_document_topics(issue_bow, minimum_probability=0.0)
                    
                    # Create full distribution vector
                    topic_probs_full = [0.0] * topics_k
                    for topic_id, prob in raw_doc_topics:
                        if topic_id < topics_k : # Ensure topic_id is within bounds
                            topic_probs_full[topic_id] = prob
                    distribution_per_issue_list.append(topic_probs_full)
                    
                    # Find dominant topic
                    if topic_probs_full: # Check if there are any probabilities
                        dominant_topic_id = topic_probs_full.index(max(topic_probs_full))
                    else: # Handle case of no topics (e.g., empty issue_bow after filtering)
                        dominant_topic_id = -1 # Or some other placeholder

                    topic_per_issue_list.append(dominant_topic_id)
                    
                    # Get top terms for dominant topic
                    if dominant_topic_id != -1:
                        top_terms = [term for term, prob in lda_model.show_topic(dominant_topic_id, topn=10)]
                    else:
                        top_terms = [] # Empty list if no dominant topic
                    topterms_per_issue_list.append(top_terms)

                # Add new features to the DataFrame copy
                current_x_data["topic_id"] = topic_per_issue_list
                current_x_data["top_terms"] = topterms_per_issue_list
                
                df_distribution = pd.DataFrame(
                    distribution_per_issue_list, 
                    columns=[f'topic_{i}' for i in range(topics_k)],
                    index=current_x_data.index # Align index with current_x_data
                )

                # Save feature DataFrames
                current_x_data.drop(current_x_data.columns[current_x_data.columns.str.contains('unnamed', case=False)], axis=1, inplace=True, errors='ignore')
                output_features_path = os.path.join(full_output_path, f"{dataset_name}_{topics_k}_topics.csv")
                current_x_data.to_csv(output_features_path, sep='\t', encoding='utf-8', index=False)
                # print(f"      Saved features to: {output_features_path}")

                df_distribution.drop(df_distribution.columns[df_distribution.columns.str.contains('unnamed', case=False)], axis=1, inplace=True, errors='ignore')
                output_dist_path = os.path.join(full_output_path, f"{dataset_name}_distribution_{topics_k}_topics.csv")
                df_distribution.to_csv(output_dist_path, sep='\t', encoding='utf-8', index=False)
                # print(f"      Saved topic distributions to: {output_dist_path}")
            print(f"    Features extracted and saved for {topics_k} topics.")

            # --- Step 7: Save Target Variables ---
        print(f"  Saving y_train and y_test for project {project_name}...")
        for dataset_name_y, current_y_data in [("y_train", y_train), ("y_test", y_test)]:
            current_y_data.drop(current_y_data.columns[current_y_data.columns.str.contains('unnamed', case=False)], axis=1, inplace=True, errors='ignore')
            output_y_path = os.path.join(full_output_path, f"{dataset_name_y}.csv")
            current_y_data.to_csv(output_y_path, sep='\t', encoding='utf-8', index=False)
            print(f"    Saved {dataset_name_y} to: {output_y_path}")

print("\nScript finished processing all files.")


Starting to process files...
 i = 1

Processing file: 3.2_AMBARI_5_assignees_processed.csv
  Project: AMBARI, Num Assignees Config: 5
  Output directory: C:\Users\hp\Desktop\Module-3-Task-assigning\data\data_output\step3_2_processed_data\4_AMBARI_5_assignees_features
  Data loaded. Training set size: 320, Test set size: 80
  Preparing LDA corpus from training data...
  Dictionary size after filtering: 45
    Training LDA model with 4 topics...


c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:719: RuntimeWarning: invalid value encountered in divide
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:733: RuntimeWarning: invalid value encountered in divide
  sstats[:, ids] += np.outer(expElogthetad.T, cts / phinorm)
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:141: RuntimeWarning: invalid value encountered in divide
  b = np.sum(gradf / q) / (1 / c + np.sum(1 / q))
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:143: RuntimeWarning: invalid value encountered in divide
  dprior = -(gradf - b) / q
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:719: RuntimeWarning: invalid value encountered in divide
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)


    Extracting features for 4 topics...


c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:1230: RuntimeWarning: invalid value encountered in divide
  return topics / topics.sum(axis=1)[:, None]
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:719: RuntimeWarning: invalid value encountered in divide
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:1230: RuntimeWarning: invalid value encountered in divide
  return topics / topics.sum(axis=1)[:, None]
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:719: RuntimeWarning: invalid value encountered in divide
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)


    Features extracted and saved for 4 topics.
    Training LDA model with 6 topics...


...

c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:719: RuntimeWarning: invalid value encountered in divide
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:1230: RuntimeWarning: invalid value encountered in divide
  return topics / topics.sum(axis=1)[:, None]
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:1230: RuntimeWarning: invalid value encountered in divide
  return topics / topics.sum(axis=1)[:, None]
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:719: RuntimeWarning: invalid value encountered in divide
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:719: RuntimeWarning: invalid value encountered in divide
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)


    Features extracted and saved for 160 topics.
    Training LDA model with 170 topics...
    Extracting features for 170 topics...


c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:733: RuntimeWarning: invalid value encountered in divide
  sstats[:, ids] += np.outer(expElogthetad.T, cts / phinorm)
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:141: RuntimeWarning: invalid value encountered in divide
  b = np.sum(gradf / q) / (1 / c + np.sum(1 / q))
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:143: RuntimeWarning: invalid value encountered in divide
  dprior = -(gradf - b) / q
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:719: RuntimeWarning: invalid value encountered in divide
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:1230: RuntimeWarning: invalid value encountered in divide
  return topics / topics.sum(axis=1)[:, None]


    Features extracted and saved for 170 topics.
  Saving y_train and y_test for project SPARK...
    Saved y_train to: C:\Users\hp\Desktop\Module-3-Task-assigning\data\data_output\step3_2_processed_data\4_SPARK_5_assignees_features\y_train.csv
    Saved y_test to: C:\Users\hp\Desktop\Module-3-Task-assigning\data\data_output\step3_2_processed_data\4_SPARK_5_assignees_features\y_test.csv

Script finished processing all files.


c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:1230: RuntimeWarning: invalid value encountered in divide
  return topics / topics.sum(axis=1)[:, None]
c:\Users\hp\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:719: RuntimeWarning: invalid value encountered in divide
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)
